In [1]:
from pathlib import Path
import os
import json
from collections import defaultdict

import numpy as np
from sklearn.model_selection import StratifiedGroupKFold
import pandas as pd
import shutil

# === 경로 설정 ===
BASE_DIR = Path("/data/ephemeral/home/pro-cv-objectdetection-cv-11-sj/dataset").resolve()
TRAIN_JSON = BASE_DIR / "train.json"
TEST_JSON = BASE_DIR / "test.json"

print("BASE_DIR exists:", BASE_DIR.exists())
print("TRAIN_JSON exists:", TRAIN_JSON.exists())
print("TEST_JSON exists:", TEST_JSON.exists())

# === train.json 로드 ===
with open(TRAIN_JSON, "r") as f:
    train_data = json.load(f)

print(
    "images:", len(train_data["images"]),
    "annotations:", len(train_data["annotations"]),
    "categories:", len(train_data["categories"])
)

# === category id -> index, index -> name 매핑 만들기 ===
categories = train_data["categories"]
cat_id_to_idx = {c["id"]: i for i, c in enumerate(categories)}  # 예: {1:0, 2:1, ...}
idx_to_name = {i: c["name"] for i, c in enumerate(categories)}  # 예: {0:"General trash", ...}
num_classes = len(categories)

print("cat_id_to_idx:", cat_id_to_idx)
print("idx_to_name:", idx_to_name)
print("num_classes:", num_classes)


BASE_DIR exists: True
TRAIN_JSON exists: True
TEST_JSON exists: True
images: 4883 annotations: 23144 categories: 10
cat_id_to_idx: {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9}
idx_to_name: {0: 'General trash', 1: 'Paper', 2: 'Paper pack', 3: 'Metal', 4: 'Glass', 5: 'Plastic', 6: 'Styrofoam', 7: 'Plastic bag', 8: 'Battery', 9: 'Clothing'}
num_classes: 10


In [2]:
# === StratifiedGroupKFold 준비 ===
annots = train_data["annotations"]

# (image_id, category_id) 쌍
var = [(ann["image_id"], ann["category_id"]) for ann in annots]
X = np.ones((len(var), 1))  # 의미 없는 placeholder
y = np.array([v[1] for v in var])       # category_id
groups = np.array([v[0] for v in var])  # image_id

cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=411)

# 한 개 fold만 골라서 사용 (예: fold 0)
fold_to_use = 0
train_ids = None
val_ids = None

for fold_idx, (train_idx, val_idx) in enumerate(cv.split(X, y, groups)):
    print(f"Fold {fold_idx}: train_idx={len(train_idx)}, val_idx={len(val_idx)}")
    if fold_idx == fold_to_use:
        train_ids = np.unique(groups[train_idx])
        val_ids = np.unique(groups[val_idx])
        break

print(f"\nUsing fold {fold_to_use}")
print("train image ids:", len(train_ids))
print("val image ids:", len(val_ids))

train_id_set = set(train_ids.tolist())
val_id_set = set(val_ids.tolist())


Fold 0: train_idx=18633, val_idx=4511

Using fold 0
train image ids: 3914
val image ids: 969


In [3]:
YOLO_ROOT = BASE_DIR / "yolo_dataset"   # 새로 만들 YOLO용 루트
IMG_DIR = YOLO_ROOT / "images"
LBL_DIR = YOLO_ROOT / "labels"

# 필요한 디렉토리들 미리 생성
for p in [
    IMG_DIR / "train",
    IMG_DIR / "val",
    IMG_DIR / "test",
    LBL_DIR / "train",
    LBL_DIR / "val"
]:
    p.mkdir(parents=True, exist_ok=True)

print("YOLO_ROOT:", YOLO_ROOT)

# image_id -> image_info
images_by_id = {img["id"]: img for img in train_data["images"]}

# image_id별 annotation 모으기
annotations_by_image = defaultdict(list)
for ann in annots:
    annotations_by_image[ann["image_id"]].append(ann)

# 원본 파일을 symlink로 연결 (안 되면 copy)
def link_or_copy(src: Path, dst: Path):
    dst.parent.mkdir(parents=True, exist_ok=True)
    if dst.exists():
        return
    try:
        os.symlink(src, dst)
    except (AttributeError, OSError, NotImplementedError):
        shutil.copy2(src, dst)


YOLO_ROOT: /data/ephemeral/home/pro-cv-objectdetection-cv-11-sj/dataset/yolo_dataset


In [4]:
def convert_split(split_name: str, image_ids):
    """
    split_name: "train" or "val"
    image_ids : 해당 split에 포함될 image_id 리스트/배열
    """
    img_split_dir = IMG_DIR / split_name
    lbl_split_dir = LBL_DIR / split_name
    img_split_dir.mkdir(parents=True, exist_ok=True)
    lbl_split_dir.mkdir(parents=True, exist_ok=True)

    n_images = 0
    n_labels = 0

    for image_id in image_ids:
        info = images_by_id[image_id]

        # COCO 표준은 "file_name" 이고, 문제 설명은 "filename" 이라서 둘 다 대응
        file_name = info.get("file_name", info.get("filename"))
        if file_name is None:
            raise KeyError("image entry missing 'file_name'/'filename'")

        # 원본 이미지 경로 (예: train/002.jpg)
        src_img = BASE_DIR / file_name
        if not src_img.exists():
            # 혹시 file_name이 "002.jpg" 형태라면
            src_img = (BASE_DIR / "train" / Path(file_name).name)
        if not src_img.exists():
            print(f"[{split_name}] WARNING: image not found for id {image_id}: {file_name}")
            continue

        base_name = Path(file_name).name  # '002.jpg'
        dst_img = img_split_dir / base_name
        link_or_copy(src_img, dst_img)
        n_images += 1

        # 이미지 크기
        width = info.get("width", 1024)
        height = info.get("height", 1024)

        # 이 이미지에 대한 annotation들
        anns = annotations_by_image.get(image_id, [])
        if not anns:
            # 객체가 하나도 없으면 라벨 파일 없어도 됨
            continue

        lines = []
        for ann in anns:
            cat_original = ann["category_id"]
            cls_idx = cat_id_to_idx[cat_original]   # 0~9 로 매핑

            # COCO bbox: [x_min, y_min, w, h] (pixel 단위)
            x_min, y_min, w, h = ann["bbox"]
            x_c = x_min + w / 2.0
            y_c = y_min + h / 2.0

            # 정규화 (0~1)
            x_c /= width
            y_c /= height
            w /= width
            h /= height

            # 혹시 범위를 벗어나는 경우 안전하게 클립
            x_c = min(max(x_c, 0.0), 1.0)
            y_c = min(max(y_c, 0.0), 1.0)
            w = min(max(w, 0.0), 1.0)
            h = min(max(h, 0.0), 1.0)

            lines.append(f"{cls_idx} {x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}")

        label_path = lbl_split_dir / (Path(base_name).stem + ".txt")
        with open(label_path, "w") as f:
            f.write("\n".join(lines))
        n_labels += 1

    print(f"[{split_name}] wrote {n_images} images and {n_labels} label files")


In [5]:
convert_split("train", train_ids)
convert_split("val", val_ids)


[train] wrote 3914 images and 3914 label files
[val] wrote 969 images and 969 label files


In [6]:
# === test.json 로드 후 images/test 로 복사 (라벨은 없음) ===
with open(TEST_JSON, "r") as f:
    test_data = json.load(f)

images_test = test_data["images"]
img_test_dir = IMG_DIR / "test"
img_test_dir.mkdir(parents=True, exist_ok=True)

test_filename_to_id = {}

for img in images_test:
    file_name = img.get("file_name", img.get("filename"))
    if file_name is None:
        continue

    src_img = BASE_DIR / file_name  # 보통 'test/3105.jpg'
    if not src_img.exists():
        src_img = (BASE_DIR / "test" / Path(file_name).name)
    if not src_img.exists():
        print(f"[test] WARNING: image not found: {file_name}")
        continue

    dst_img = img_test_dir / Path(file_name).name  # '3105.jpg'
    link_or_copy(src_img, dst_img)

    # 나중에 submission 만들 때 image_id를 찾기 위한 매핑 (예: 'test/3105.jpg' -> id)
    test_filename_to_id[file_name] = img["id"]

print("Copied test images:", len(list(img_test_dir.glob("*.*"))))


Copied test images: 4871


In [7]:
data_yaml_path = YOLO_ROOT / "trash10.yaml"

names = [
    "General trash",
    "Paper",
    "Paper pack",
    "Metal",
    "Glass",
    "Plastic",
    "Styrofoam",
    "Plastic bag",
    "Battery",
    "Clothing",
]

with open(data_yaml_path, "w") as f:
    f.write(f"path: {YOLO_ROOT}\n")
    f.write("train: images/train\n")
    f.write("val: images/val\n")
    f.write("test: images/test\n")  # test는 라벨 없이 inference용
    f.write("names:\n")
    for i, name in enumerate(names):
        f.write(f"  {i}: {name}\n")

print("=== trash10.yaml ===")
print(data_yaml_path.read_text())


=== trash10.yaml ===
path: /data/ephemeral/home/pro-cv-objectdetection-cv-11-sj/dataset/yolo_dataset
train: images/train
val: images/val
test: images/test
names:
  0: General trash
  1: Paper
  2: Paper pack
  3: Metal
  4: Glass
  5: Plastic
  6: Styrofoam
  7: Plastic bag
  8: Battery
  9: Clothing



In [ ]:
# 사용 불가능한 코드입니다.
# metrics = model.val()  # train 때 사용한 data/imgsz 등을 기억하고 있음
# print("mAP50-95:", metrics.box.map)
# print("mAP50:", metrics.box.map50)
# print("mAP75:", metrics.box.map75)


Ultralytics 8.3.235 🚀 Python-3.10.13 torch-2.1.0+cu118 CUDA:0 (Tesla V100-SXM2-32GB, 32501MiB)
YOLO11n summary (fused): 100 layers, 2,584,102 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1822.7±866.3 MB/s, size: 169.8 KB)
val: Scanning /data/ephemeral/home/pro-cv-objectdetection-cv-11-sj/dataset/yolo_dataset/labels/val.cache... 969 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 969/969 1.4Mit/s 0.0s0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 61/61 4.5it/s 13.6s0.1s
                   all        969       4511      0.587      0.489      0.513      0.418
         General trash        438        805      0.475      0.272      0.292      0.229
                 Paper        335       1237      0.579      0.417      0.474      0.314
            Paper pack        130        191      0.557      0.381      0.417      0.351
                 Metal        109        167       0.52      